# Week 1 Graded Quiz

In [1]:
import pandas as pd
import edhec_risk_kit as erk
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Question 1

In [9]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["1991":]

In [10]:
ind49_rets.tail()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
2018-08,0.0715,0.0114,-0.0308,-0.0186,-0.0543,0.0738,0.0250,-0.0463,0.0254,0.0526,...,-0.0062,0.0291,0.0366,0.0911,0.0364,0.0277,0.0352,-0.0423,0.0004,0.0333
2018-09,-0.0392,-0.0220,0.0310,0.0019,0.0537,0.0717,-0.0075,-0.0209,0.0107,0.0130,...,0.0189,0.0105,-0.0148,0.0061,0.0251,-0.0282,0.0116,-0.0492,-0.0340,0.0068
2018-10,-0.0197,0.0095,0.0118,-0.0157,0.0790,-0.1504,-0.1605,-0.0569,-0.0051,-0.1014,...,-0.1100,-0.0975,-0.0788,-0.1021,-0.0171,-0.0613,-0.0413,-0.1213,-0.0455,-0.0515
2018-11,-0.0357,0.0029,0.0684,0.0579,-0.0843,-0.0715,-0.0001,0.0375,0.0644,-0.0099,...,0.1021,0.0617,0.0318,0.0159,0.0616,0.0292,0.0432,0.0715,0.0006,0.0355
2018-12,-0.1093,-0.1093,-0.0783,-0.0969,-0.1580,-0.1635,-0.0750,-0.1442,-0.0557,-0.0728,...,-0.1248,-0.1519,-0.1121,-0.0919,-0.0628,-0.1279,-0.0985,-0.1172,-0.0874,-0.0654


In [11]:
fff = erk.get_fff_returns()["1991":]
fff.head()

,Mkt-RF,SMB,HML,RF
1991-01,0.0469,0.0379,-0.0184,0.0052
1991-02,0.0719,0.0395,-0.0054,0.0048
1991-03,0.0265,0.0389,-0.0123,0.0044
1991-04,-0.0028,0.0050,0.0142,0.0053
1991-05,0.0365,-0.0034,-0.0057,0.0047


In [12]:
import statsmodels.api as sm

ind49_excess = ind49_rets["1991":] - fff.loc["1991":, ['RF']].values
mkt_excess = fff.loc["1991":,['Mkt-RF']]
ind49_excess.shape, mkt_excess.shape

((336, 49), (336, 1))

In [13]:
exp_var = mkt_excess.copy()
exp_var["Constant"] = 1
exp_var.shape

(336, 2)

In [32]:
sm.OLS(ind49_excess[:]["Beer"], exp_var[:]).fit().params.round(2)

Mkt-RF      0.53
Constant    0.00
dtype: float64

## Question 2

In [33]:
sm.OLS(ind49_excess[:]["Steel"], exp_var[:]).fit().params.round(2)

Mkt-RF      1.55
Constant   -0.01
dtype: float64

## Question 3

In [50]:
sm.OLS(ind49_excess["2013":]["Beer"], exp_var["2013":]).fit().params.round(2)

Mkt-RF      0.59
Constant    0.00
dtype: float64

## Question 4

In [51]:
sm.OLS(ind49_excess["2013":]["Steel"], exp_var["2013":]).fit().params.round(2)

Mkt-RF      1.42
Constant   -0.01
dtype: float64

## Question 5

In [90]:
tmp = {}
for name in ind49_excess.columns:
    tmp[name] = sm.OLS(ind49_excess["1991":"1993"][name], exp_var["1991":"1993"]).fit().params.iloc[0]
max(tmp, key=tmp.get)

'Hlth'

## Question 6

In [91]:
tmp = {}
for name in ind49_excess.columns:
    tmp[name] = sm.OLS(ind49_excess["1991":"1993"][name], exp_var["1991":"1993"]).fit().params.iloc[0]
min(tmp, key=tmp.get)

'Gold'

## Question 7

In [97]:
exp_var2 = exp_var.copy()
exp_var2["HML"] = fff.loc["1991":,['HML']]
exp_var2["SMB"] = fff.loc["1991":,['SMB']]
exp_var2.head()

,Mkt-RF,Constant,HML,SMB
1991-01,0.0469,1,-0.0184,0.0379
1991-02,0.0719,1,-0.0054,0.0395
1991-03,0.0265,1,-0.0123,0.0389
1991-04,-0.0028,1,0.0142,0.0050
1991-05,0.0365,1,-0.0057,-0.0034


In [105]:
# Test regression
sm.OLS(ind49_excess[:]["Beer"], exp_var2[:]).fit().params.iloc[3].round(2)

-0.36

In [107]:
tmp2 = {}
for name in ind49_excess.columns:
    tmp2[name] = sm.OLS(ind49_excess[:][name], exp_var2[:]).fit().params.iloc[3].round(2)
max(tmp2, key=tmp2.get)

'FabPr'

## Question 8

In [108]:
min(tmp2, key=tmp2.get)

'Beer'

## Question 9

In [111]:
# Test regression
sm.OLS(ind49_excess[:]["Beer"], exp_var2[:]).fit().params.iloc[2].round(2)

0.01

In [112]:
tmp3 = {}
for name in ind49_excess.columns:
    tmp3[name] = sm.OLS(ind49_excess[:][name], exp_var2[:]).fit().params.iloc[2].round(2)
max(tmp3, key=tmp3.get)

'Txtls'

## Question 9

In [114]:
min(tmp3, key=tmp3.get)

'Softw'